In [1]:
import keypress_recognition.models as models
from keypress_recognition import dataset_p as dataset
import numpy as np
import matplotlib.pyplot as plt
import cv2
%load_ext autoreload
%autoreload 2

In [2]:
concat = False
delta = 3 if concat else 1
channels = 3 * delta

In [4]:
mod = {}
mod['black'] = models.get_black_bundle_model(channels)
mod['black'].load_model('keypress_recognition/models/best_black_bundle.tar')
mod['white'] = models.get_white_bundle_model(channels)
mod['white'].load_model('keypress_recognition/models/best_white_bundle.tar') 

Trying to load model from keypress_recognition/models/best_black_bundle.tar ...
done
Trying to load model from keypress_recognition/models/best_white_bundle.tar ...
done


In [5]:
dataset.load_all_data(color=['black', 'white'], size=['bundle'], keypress=True, concatenate=concat, delta=delta)

Start seperating keyboard ...
  White single width: 21px
  Black single width: 20px
  White bundle width: 37px
  Black bundle width: 36px


IntProgress(value=0, max=14273)

In trainset: 
  # of pressed black key: 8755
  # of unpressed black key: 44141
  # of pressed white key: 19083
  # of unpressed white key: 57799


IntProgress(value=0, max=5000)

In testset: 
  # of pressed black key: 761
  # of unpressed black key: 22531
  # of pressed white key: 10831
  # of unpressed white key: 23055


IntProgress(value=0, max=2993)

In valset: 
  # of pressed black key: 817
  # of unpressed black key: 10697
  # of pressed white key: 5090
  # of unpressed white key: 11944


In [6]:
def visualize_keys(note):
    """
    note: bool
    return: str
    """
    return ("Y" if note else "N")

def display_concat_img(img, idx=None):
    if concat:
        if idx is None:
            return img[:, :, :, 3:6]
        else:
            return img[idx, :, :, 3:6]
    if idx is None:
        return img
    else:
        return img[idx, :, :, :]


In [7]:
for color in ('black', 'white'):
    results = mod[color].test(dataset, size='bundle', color=color, concatenate=concat)
    for k, v in results.items():
        print("{:10}: {:.4f}".format(k.title(), v))

IntProgress(value=0, max=45062)

Precision : 0.9877
Recall    : 0.9948
Accuracy  : 0.9912
F1 Score  : 0.9912


IntProgress(value=0, max=46110)

Precision : 0.9574
Recall    : 0.9706
Accuracy  : 0.9637
F1 Score  : 0.9639


In [ ]:
pred = {}
for color in ('black', 'white'):
    inputs = np.transpose(dataset.X['bundle'][color]['test'], (0, 3, 1, 2))
    print(inputs.shape)
    pred[color] = mod[color].evaluate(inputs)
    pred[color] = pred[color].numpy()
    np.save(f'{color}_prediction.npy', pred[color])